In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
import time

In [3]:
pickle_file = 'SVHN_1x64x64_train.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    del save  # hint to help gc free up memory
    print('train set', train_dataset.shape, train_labels.shape)

c3 = []

for i in range(97722):
    categ = train_labels[i][0]
    if(categ == 2):
        c3.append(i)

print(len(c3))

train set (97722, 1, 64, 64) (97722, 6)
40006


In [4]:
c3_data = train_dataset[c3]
c3_target = train_labels[c3]
print(c3_data.shape)
print(c3_target.shape)

(40006, 1, 64, 64)
(40006, 6)


In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 3, padding=(1, 1))
        self.conv2 = nn.Conv2d(20, 40, 3, padding=(1, 1))
        self.conv3 = nn.Conv2d(40, 80, 3, padding=(1, 1))
        self.conv4 = nn.Conv2d(80, 120, 3, padding=(1, 1))
        self.conv5 = nn.Conv2d(120, 160, 3, padding=(1, 1))
        self.conv6 = nn.Conv2d(160, 200, 3, padding=(1, 1))
        self.conv7 = nn.Conv2d(200, 240, 3, padding=(1, 1))
        self.pool = nn.MaxPool2d(2, 2)
        self.FC = nn.Linear(960, 1080)
        self.digitlength = nn.Linear(1080, 7)
        self.digit1 = nn.Linear(1080, 10)
        self.digit2 = nn.Linear(1080, 10)
        self.digit3 = nn.Linear(1080, 10)
        self.digit4 = nn.Linear(1080, 10)
        self.digit5 = nn.Linear(1080, 10)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.pool(F.relu(self.conv4(x)))
        x = self.pool(F.relu(self.conv5(x)))
        x = self.pool(F.relu(self.conv6(x)))
        x = self.pool(F.relu(self.conv7(x)))
        x = x.view(-1, 960)
        x = self.FC(x)
        yl = self.digitlength(x)
        y1 = self.digit1(x)
        y2 = self.digit2(x)
        y3 = self.digit3(x)
        y4 = self.digit4(x)
        y5 = self.digit5(x)
        return [yl, y1, y2, y3, y4, y5]

In [6]:
net = Net()
f = open('c3_leg_ep_20.pkl', 'rb')
net.load_state_dict(torch.load(f))
f.close()
net.cuda()

Net (
  (conv1): Conv2d(1, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(20, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(40, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(80, 120, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(120, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(160, 200, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv7): Conv2d(200, 240, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (FC): Linear (960 -> 1080)
  (digitlength): Linear (1080 -> 7)
  (digit1): Linear (1080 -> 10)
  (digit2): Linear (1080 -> 10)
  (digit3): Linear (1080 -> 10)
  (digit4): Linear (1080 -> 10)
  (digit5): Linear (1080 -> 10)
)

In [7]:
for param in net.parameters():
    if(param.grad is not None):
        print(param)

In [8]:
c3_data_tensor = torch.from_numpy(c3_data)
c3_target_tensor = torch.from_numpy(c3_target).type(torch.LongTensor)
print(c3_data_tensor.type(), c3_data_tensor.size())
print(c3_target_tensor.type(), c3_target_tensor.size())

torch.FloatTensor torch.Size([40006, 1, 64, 64])
torch.LongTensor torch.Size([40006, 6])


In [9]:
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [10]:
num_epochs = 20
batch_size = 64
num_train =  c3_data.shape[0]
iter_per_epoch = num_train // batch_size
print_every = 300
print(iter_per_epoch)

625


In [11]:
epoch_losses = {i:[] for i in range(num_epochs)}
loss_history = []

In [12]:
for epoch in range(num_epochs):  # loop over the dataset multiple times
    
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 110)

    i = 0
    rng_state = torch.get_rng_state()
    new_idxs = torch.randperm(num_train)
    c3_X = c3_data_tensor[new_idxs]
    c3_Y = c3_target_tensor[new_idxs]
    
    t1 = time.time()
    for t in range(iter_per_epoch):
        
        X_batch = c3_X[i: i+batch_size]
        Y_batch = c3_Y[i: i+batch_size][:,0:4]
        i += batch_size

        Y_batch = Variable(Y_batch.cuda())
        X_batch = Variable(X_batch.cuda())
        
        optimizer.zero_grad()
        
        outputs = net(X_batch)
        
        lossl = objective(outputs[0], Y_batch[:, 0])
        loss1 = objective(outputs[1], Y_batch[:, 1])
        loss2 = objective(outputs[2], Y_batch[:, 2])
        final_loss = lossl + loss1 + loss2
        
        final_loss.backward()
        
        optimizer.step()
        
        loss_history.append(final_loss.data[0])
        epoch_losses[epoch].append(final_loss.data[0])
        
        if (t % print_every == 0):
            print('Iteration : ', t+1, ' / ', iter_per_epoch)
            print('loss : ', final_loss.data[0])
            print('lossl : ', lossl.data[0], 'loss1 : ', loss1.data[0], 'loss2 : ', loss2.data[0])
    t2 = time.time()
    print("time taken : ", t2-t1)
    print('-' * 110)
        

Epoch 0/19
--------------------------------------------------------------------------------------------------------------
Iteration :  1  /  625
loss :  93.4910888671875
lossl :  84.45161437988281 loss1 :  0.5001339316368103 loss2 :  8.539336204528809
Iteration :  301  /  625
loss :  0.7808812856674194
lossl :  0.000708363950252533 loss1 :  0.35415735840797424 loss2 :  0.42601555585861206
Iteration :  601  /  625
loss :  0.540520966053009
lossl :  0.0006491988897323608 loss1 :  0.22309759259223938 loss2 :  0.3167741894721985
time taken :  979.2570998668671
--------------------------------------------------------------------------------------------------------------
Epoch 1/19
--------------------------------------------------------------------------------------------------------------
Iteration :  1  /  625
loss :  0.14915671944618225
lossl :  0.00018310546875 loss1 :  0.07522555440664291 loss2 :  0.07374805212020874
Iteration :  301  /  625
loss :  0.34971943497657776
lossl :  0.00036

KeyboardInterrupt: 

In [14]:
f = open("c3-c2_ep_2.pkl", "bw")
torch.save(net.state_dict(), f)
f.close()

In [15]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.figure()
plt.plot(loss_history)

/home/sreekar/miniconda3/envs/deep/lib/python3.5/site-packages/matplotlib/font_manager.py:280: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '
